In [8]:
import os
import pandas as pd

# 데이터 파일들이 있는 폴더 경로 설정
data_folder = "./data/"

# 폴더가 존재하는지 확인
df_folder_exists = os.path.exists(data_folder)
if not df_folder_exists:
    raise FileNotFoundError(f"지정된 폴더 경로가 존재하지 않습니다: {data_folder}")

# 폴더 내 모든 CSV 파일을 불러와서 결합하기
all_files = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.endswith('.csv')]

# 모든 데이터를 하나의 DataFrame으로 결합
dataframes = []
for file in all_files:
    try:
        df = pd.read_csv(file)
        dataframes.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

combined_df = pd.concat(dataframes, ignore_index=True)

# 결합된 데이터 확인 및 저장
print(f"총 데이터 수: {len(combined_df)}")
combined_df.to_csv("./data/combined_reviews.csv", index=False)

# 간단한 데이터 전처리
def preprocess_reviews(df):
    # 리뷰 컬럼 이름 확인 후 사용 가능한 컬럼 찾기
    possible_columns = ['리뷰', 'review', '내용', 'Review']
    text_column = None
    for column in possible_columns:
        if column in df.columns:
            text_column = column
            break
    
    if text_column is None:
        raise KeyError("리뷰 텍스트 컬럼을 찾을 수 없습니다. 컬럼 이름을 확인해주세요.")
    
    # 결측치 제거
    df = df.dropna(subset=[text_column])
    
    # 리뷰 텍스트 전처리 (간단하게 특수문자 제거 예시)
    df.loc[:, text_column] = df[text_column].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
    return df

# 전처리 실행
processed_df = preprocess_reviews(combined_df)

# 전처리된 데이터 저장
processed_df.to_csv("./data/processed_reviews.csv", index=False)

print("데이터 전처리가 완료되었습니다.")


총 데이터 수: 2975
데이터 전처리가 완료되었습니다.


/tmp/ipykernel_4092/488302634.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, text_column] = df[text_column].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)


1. 데이터 준비 및 라벨링
라벨링 작업: 감성 분석을 수행하려면 리뷰 데이터에 긍정/부정 라벨이 필요합니다. 리뷰 데이터를 수동 또는 자동으로 라벨링할 수 있어요. 자동 라벨링을 위해 감정 표현이 명확한 단어를 기준으로 긍정/부정을 판별하거나, 일부 샘플을 수동으로 라벨링 후 이를 기반으로 나머지를 예측할 수도 있습니다.
목표 컬럼 설정: 전처리된 데이터에서 감성 분석에 필요한 텍스트와 레이블(긍정/부정)을 분리합니다.

3. 텍스트 전처리
토큰화 및 정규화: 리뷰 텍스트를 텍스트 토큰(token)으로 분리하고 정규화하는 작업이 필요합니다. 예를 들어, 형태소 분석기(예: konlpy의 Okt 등)를 사용해 한국어 리뷰를 분석할 수 있어요.
불용어 제거: 의미 없는 단어(예: 조사, 일반적인 불용어 등)를 제거합니다.


5. 데이터 벡터화
텍스트 데이터를 머신러닝 모델에 입력하기 위해 벡터화가 필요합니다. 이를 위해 TF-IDF 또는 워드 임베딩(예: Word2Vec, FastText, BERT 등)을 사용할 수 있습니다.


7. 모델 학습
학습 데이터와 테스트 데이터 분할: 데이터를 학습과 평가를 위해 분할합니다 (예: 80% 학습, 20% 테스트).
모델 선택: 감성 분석에 사용할 모델을 선택합니다. 간단하게는 로지스틱 회귀나 Naive Bayes 같은 모델을 사용할 수 있으며, 딥러닝 모델로는 LSTM이나 BERT를 사용할 수 있습니다.
모델 학습: 준비된 데이터를 사용하여 모델을 학습시킵니다.


9. 모델 평가
테스트 데이터를 사용하여 학습된 모델을 평가합니다.
정확도(Accuracy), 정밀도(Precision), 재현율(Recall), F1 Score 등의 지표를 확인해 모델의 성능을 평가합니다.


11. 감성 분석 수행
학습된 모델을 사용해 특정 가게의 리뷰에 대한 감성 분석을 수행하고, 결과를 시각화하거나 요약하는 작업을 진행합니다.
위의 단계들 중에서 다음으로 데이터 라벨링 및 추가 전처리를 진행하는 것이 좋습니다. 만약 데이터 라벨링에 대해 도움 필요하시거나 모델 학습에 대해 알고 싶으시면, 더 자세히 설명드리겠습니다. 어떻게 진행하고 싶으신가요?

# okt

In [9]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 간단한 데이터 전처리
def preprocess_reviews(df):
    # 리뷰 컬럼 이름 확인 후 사용 가능한 컬럼 찾기
    possible_columns = ['리뷰', 'review', '내용', 'Review']
    text_column = None
    for column in possible_columns:
        if column in df.columns:
            text_column = column
            break
    
    if text_column is None:
        raise KeyError("리뷰 텍스트 컬럼을 찾을 수 없습니다. 컬럼 이름을 확인해주세요.")
    
    # 결측치 제거
    df = df.dropna(subset=[text_column])
    
    # 리뷰 텍스트 전처리 (간단하게 특수문자 제거 예시)
    df.loc[:, text_column] = df[text_column].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
    
    # 형태소 분석을 통한 리뷰 텍스트 토큰화
    df['tokenized'] = df[text_column].apply(lambda x: okt.morphs(x, stem=True))
    
    # 라벨링 작업: 긍정/부정 라벨링 - 감정 단어 기반 라벨링 (긍정 단어/부정 단어 리스트 사용)
    positive_words = [
        "좋", "훌륭", "최고", "맛있", "추천", "즐겁", "기쁘", "만족", "감동", "깨끗", "재밌", "친절",
        "감사", "좋아하", "편안", "행복", "깔끔", "존맛", "JMT", "맛집", "짱", "베스트", "대박",
        "최애", "힐링", "감탄", "인생", "역대급", "완벽", "신선", "든든", "엄지 척", "배려", "뿌듯",
        "포근", "추천템", "안심", "만족도 최고", "편리", "세련", "유용", "아늑", "기대 이상", "고급지다",
        "센스있", "특별", "화려", "러블리", "맛보장", "행운", "예쁘다", "베리 굿", "가성비 좋", "적극 추천",
        "감성적", "취향저격", "레전드", "완소", "쏘 스윗", "찐행복", "갓성비", "귀엽", "최애템", "핵꿀잼",
        "고급미", "갓벽", "미쳤다", "오져", "대만족", "찐사랑", "신박", "고퀄", "강추", "존좋", "너무 좋",
        "죽인다", "굿굿", "금손", "소확행", "최상급", "훈훈", "신세계", "고급짐", "재구매 의사", "초강추",
        "귀엽고 예쁨", "찐템", "꿀템", "신박하다", "행복함", "안심됨", "극호", "꾸안꾸", "손맛 좋",
        "핵감동", "친근감", "든든함", "사랑스러움", "행복사", "찐최애", "만점템", "찐인생템", "눈호강",
        "청정구역", "맑고 깨끗", "깨알재미", "귀염", "사랑스럽다", "가성비템", "깔쌈", "베리굿", "딱 좋음",
        "잘 나감", "편의성 굿", "안정적", "배부름", "센스 만점", "짱짱맨", "적극적", "말해 뭐해", "넘사벽",
        "신뢰감", "꿀잼보장", "심쿵", "센스 넘침", "재미짐", "편리함", "만족스러움", "무한 감동", "탄탄함",
        "우수함", "굿", "존맛탱구리", "ㄴㅇㅅ", "내돈내산", "또방문", "재방문 의사 있", "유명", "소문낼",
        "유명한 이유가 있", "돈 값을 하는", "충족", "ㄱㅇㄷ", "개이득", "최상", "킹왕짱", "행복함",
        "맛집 인정", "찐맛", "호불호 없음", "득템각", "부담없", "소장각", "갓템", "마음에 쏙", "신뢰성",
        "마음에 든다", "푹신함", "대존맛", "신의 한 수",  "극찬",
        "알차다", "취향존중", "무조건 추천", "고급", "실속 있음", "잘 맞음", "너무 예쁘다",
        "믿고 구매", "만족감", "쓸모있음", "매력있음", "비교불가", "퀄리티 갑", "최상의 선택", "역시 좋다", "손이 가요",
        "가치 있음", "핫플", "핵추천", "혜자", "품질보장", "끝내줌", "강력 추천", "금방 동남",
        "엄지척", "거의 완벽", "찬사받음", "감동적", "눈이 즐거움", "존예", 
        "정말 좋아요", "불만 없음", "깔끔한 마무리", "퀄리티 짱", "세상 최고", "매력적", "끝내줌", "눈물남",
        "현존 최고", "좋은 선택", "짱짱", "상상초월", "미친 가성비", "저렴한데 좋음", "찐", "돈 값함",
        "인정", "핵만족", "보장된 품질", "칭찬할 만함", "행운템", "아기자기함", "다시 가고 싶음",
        "핵감사", "짱짱짱", "여긴 진짜다"
    ]

    negative_words = [
        "나쁘", "별로", "싫", "최악", "불편", "문제", "실망", "불만", "안좋", "안되", "부족", "힘들",
        "비싸", "짜증", "불친절", "후회", "아쉽", "형편없", "엉망", "답답", "지루", "불쾌", "피곤",
        "귀찮", "못하", "어렵", "복잡", "짜증나", "실수", "기대 이하", "아깝", "화나", "무례", "미흡",
        "고통", "시끄럽", "안타깝", "낡", "지저분", "아프", "헛걸음", "속상", "똥손", "후지", "망",
        "버거움", "별로임", "바가지", "낙후", "불량", "헛돈", "극혐", "대충", "불쾌한 경험",
        "비추", "별로네", "허접", "노답", "구려", "폐급", "구리다", "짜증 폭발", "돈낭비", "헬", "멘붕",
        "분노", "허접하다", "불편함", "답답함", "현타", "먹튀", "짜증남", "신경쓰임", "헛돈", "개노답",
        "한심", "쌩돈", "쓰레기", "이딴", "어설프다", "오류 많음", "형편없음", "허무함", "낭비", "아쉬움",
        "개판", "헬게이트", "어이없음", "대실망", "구림", "후회막급", "서비스 엉망", "좌절", "불편하다",
        "무리수", "취소각", "짜증만땅", "불만족", "더럽다", "화가남", "찝찝", "바가지", "속터짐", "별로다",
        "실망스러움", "별로에요", "스트레스", "무책임", "노잼", "먹금", "못난", "오바임", "과대광고",
        "구라", "실망임", "불만족스럽", "개노답", "망했", "못하겠", "허접쓰", "오지랖", "지저분함",
        "헛소리", "오류투성이", "피곤함", "서비스 엉망진창", "예민함", "답없다", "과함", "잔소리",
        "번거로움", "불완전", "뻘짓", "정떨어짐", "영혼 없음", "허술함", "제자리", "정리안됨", "답이 없음",
        "빈약함", "불친절한 응대", "뻔뻔", "낙후됨", "어이상실", "시간낭비", "멀미", "부실함", "막장임",
        "헛고생", "최악의 경험", "실망감", "엉망진창", "걱정됨", "재방문 의사 없", "개노맛", "실망각",
        "차라리 안 샀음", "먹다 버림", "재구매 없음", "헛돈 날림", "버림", "답답함", "후회함", "개망",
        "터무니없", "형편 없음", "퀄리티 떨어짐", "돈 아까움", "상태 불량", "비효율적", "쓸모없음",
        "버림", "품질 저하", "오바스러움", "가성비 별로", "마음에 안 듦", "손해", "완전 별로", "무의미",
        "효율성 없음", "가치 없음", "엉성함", "재방문 안함", "비추함", "믿고 걸러", "두 번 다시", "안 맞음",
        "어이없다", "구매 비추천", "완전 구림", "불량품", "기대와 다름", "불쾌감", "미스", "절대 비추",
        "낭비된 돈", "비추요", "이 돈값 못함", "싼 게 비지떡", "텅텅", "돈값 못함", "손해봤다",
        "별로였음", "너무 별로", "돈 아까움", "실망", "안간다", "차라리 딴데", "속았음",
        "후회만 남음", "맘 상함", "한 번 가고 안 감", "사기템", "버림", "쓰레기템", "더러움",
        "터무니 없음", "낚임", "두 번은 안 감", "쓰레기 서비스", "불안함", "서비스 미숙", "당황스러움",
        "너무 느림", "후회만 가득", "갈 이유 없음", "마음 상함", "말도 안 됨", "너무 허접", "없어서 못감",
        "체감상 별로", "기분 상함", "하자 많음", "최악의 결정", "구매 후회", "안습", "저렴한 티", "헛된 시간",
        "망함", "지저분", "두 번 안 감", "손해봄", "싼 맛에 샀는데 후회", "기대보다 별로", "차라리 안 사는 게",
        "기분 상함", "아무것도 아님", "안 사는 게 나음", "갓성비 아님", "여기만 빼고 다 좋음", "진절머리",
        "진저리", "별로다 못해 최악", "위생이 별로"
    ]
    
    def label_review(tokens):
        if any(word in tokens for word in positive_words):
            return '긍정'
        elif any(word in tokens for word in negative_words):
            return '부정'
        else:
            return '중립'
    
    df['label'] = df['tokenized'].apply(label_review)
    
    return df, text_column

# 전처리된 데이터 저장
print("데이터 전처리가 완료되었습니다.")


데이터 전처리가 완료되었습니다.


# mecab

In [ ]:
import pandas as pd
import numpy as np
from konlpy.tag import Mecab

# 형태소 분석기 초기화
mecab = Mecab()

# 간단한 데이터 전처리
def preprocess_reviews(df):
    # 리뷰 컬럼 이름 확인 후 사용 가능한 컬럼 찾기
    possible_columns = ['리뷰', 'review', '내용', 'Review']
    text_column = None
    for column in possible_columns:
        if column in df.columns:
            text_column = column
            break
    
    if text_column is None:
        raise KeyError("리뷰 텍스트 컬럼을 찾을 수 없습니다. 컬럼 이름을 확인해주세요.")
    
    # 결측치 제거
    df = df.dropna(subset=[text_column])
    
    # 리뷰 텍스트 전처리 (간단하게 특수문자 제거 예시)
    df.loc[:, text_column] = df[text_column].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
    
    # 형태소 분석을 통한 리뷰 텍스트 토큰화
    df['tokenized'] = df[text_column].apply(lambda x: mecab.morphs(x))
    
    # 라벨링 작업: 긍정/부정 라벨링 - 감정 단어 기반 라벨링 (긍정 단어/부정 단어 리스트 사용)
    positive_words = [
        "좋", "훌륭", "최고", "맛있", "추천", "즐겁", "기쁘", "만족", "감동", "깨끗", "재밌", "친절",
        "감사", "좋아하", "편안", "행복", "깔끔", "존맛", "JMT", "맛집", "짱", "베스트", "대박",
        "최애", "힐링", "감탄", "인생", "역대급", "완벽", "신선", "든든", "엄지 척", "배려", "뿌듯",
        "포근", "추천템", "안심", "만족도 최고", "편리", "세련", "유용", "아늑", "기대 이상", "고급지다",
        "센스있", "특별", "화려", "러블리", "맛보장", "행운", "예쁘다", "베리 굿", "가성비 좋", "적극 추천",
        "감성적", "취향저격", "레전드", "완소", "쏘 스윗", "찐행복", "갓성비", "귀엽", "최애템", "핵꿀잼",
        "고급미", "갓벽", "미쳤다", "오져", "대만족", "찐사랑", "신박", "고퀄", "강추", "존좋", "너무 좋",
        "죽인다", "굿굿", "금손", "소확행", "최상급", "훈훈", "신세계", "고급짐", "재구매 의사", "초강추",
        "귀엽고 예쁨", "찐템", "꿀템", "신박하다", "행복함", "안심됨", "극호", "꾸안꾸", "손맛 좋",
        "핵감동", "친근감", "든든함", "사랑스러움", "행복사", "찐최애", "만점템", "찐인생템", "눈호강",
        "청정구역", "맑고 깨끗", "깨알재미", "귀염", "사랑스럽다", "가성비템", "깔쌈", "베리굿", "딱 좋음",
        "잘 나감", "편의성 굿", "안정적", "배부름", "센스 만점", "짱짱맨", "적극적", "말해 뭐해", "넘사벽",
        "신뢰감", "꿀잼보장", "심쿵", "센스 넘침", "재미짐", "편리함", "만족스러움", "무한 감동", "탄탄함",
        "우수함", "굿", "존맛탱구리", "ㄴㅇㅅ", "내돈내산", "또방문", "재방문 의사 있", "유명", "소문낼",
        "유명한 이유가 있", "돈 값을 하는", "충족", "ㄱㅇㄷ", "개이득", "최상", "킹왕짱", "행복함",
        "맛집 인정", "찐맛", "호불호 없음", "득템각", "부담없", "소장각", "갓템", "마음에 쏙", "신뢰성",
        "마음에 든다", "푹신함", "대존맛", "신의 한 수",  "극찬",
        "알차다", "취향존중", "무조건 추천", "고급", "실속 있음", "잘 맞음", "너무 예쁘다",
        "믿고 구매", "만족감", "쓸모있음", "매력있음", "비교불가", "퀄리티 갑", "최상의 선택", "역시 좋다", "손이 가요",
        "가치 있음", "핫플", "핵추천", "혜자", "품질보장", "끝내줌", "강력 추천", "금방 동남",
        "엄지척", "거의 완벽", "찬사받음", "감동적", "눈이 즐거움", "존예", 
        "정말 좋아요", "불만 없음", "깔끔한 마무리", "퀄리티 짱", "세상 최고", "매력적", "끝내줌", "눈물남",
        "현존 최고", "좋은 선택", "짱짱", "상상초월", "미친 가성비", "저렴한데 좋음", "찐", "돈 값함",
        "인정", "핵만족", "보장된 품질", "칭찬할 만함", "행운템", "아기자기함", "다시 가고 싶음",
        "핵감사", "짱짱짱", "여긴 진짜다"
    ]

    negative_words = [
        "나쁘", "별로", "싫", "최악", "불편", "문제", "실망", "불만", "안좋", "안되", "부족", "힘들",
        "비싸", "짜증", "불친절", "후회", "아쉽", "형편없", "엉망", "답답", "지루", "불쾌", "피곤",
        "귀찮", "못하", "어렵", "복잡", "짜증나", "실수", "기대 이하", "아깝", "화나", "무례", "미흡",
        "고통", "시끄럽", "안타깝", "낡", "지저분", "아프", "헛걸음", "속상", "똥손", "후지", "망",
        "버거움", "별로임", "바가지", "낙후", "불량", "헛돈", "극혐", "대충", "불쾌한 경험",
        "비추", "별로네", "허접", "노답", "구려", "폐급", "구리다", "짜증 폭발", "돈낭비", "헬", "멘붕",
        "분노", "허접하다", "불편함", "답답함", "현타", "먹튀", "짜증남", "신경쓰임", "헛돈", "개노답",
        "한심", "쌩돈", "쓰레기", "이딴", "어설프다", "오류 많음", "형편없음", "허무함", "낭비", "아쉬움",
        "개판", "헬게이트", "어이없음", "대실망", "구림", "후회막급", "서비스 엉망", "좌절", "불편하다",
        "무리수", "취소각", "짜증만땅", "불만족", "더럽다", "화가남", "찝찝", "바가지", "속터짐", "별로다",
        "실망스러움", "별로에요", "스트레스", "무책임", "노잼", "먹금", "못난", "오바임", "과대광고",
        "구라", "실망임", "불만족스럽", "개노답", "망했", "못하겠", "허접쓰", "오지랖", "지저분함",
        "헛소리", "오류투성이", "피곤함", "서비스 엉망진창", "예민함", "답없다", "과함", "잔소리",
        "번거로움", "불완전", "뻘짓", "정떨어짐", "영혼 없음", "허술함", "제자리", "정리안됨", "답이 없음",
        "빈약함", "불친절한 응대", "뻔뻔", "낙후됨", "어이상실", "시간낭비", "멀미", "부실함", "막장임",
        "헛고생", "최악의 경험", "실망감", "엉망진창", "걱정됨", "재방문 의사 없", "개노맛", "실망각",
        "차라리 안 샀음", "먹다 버림", "재구매 없음", "헛돈 날림", "버림", "답답함", "후회함", "개망",
        "터무니없", "형편 없음", "퀄리티 떨어짐", "돈 아까움", "상태 불량", "비효율적", "쓸모없음",
        "버림", "품질 저하", "오바스러움", "가성비 별로", "마음에 안 듦", "손해", "완전 별로", "무의미",
        "효율성 없음", "가치 없음", "엉성함", "재방문 안함", "비추함", "믿고 걸러", "두 번 다시", "안 맞음",
        "어이없다", "구매 비추천", "완전 구림", "불량품", "기대와 다름", "불쾌감", "미스", "절대 비추",
        "낭비된 돈", "비추요", "이 돈값 못함", "싼 게 비지떡", "텅텅", "돈값 못함", "손해봤다",
        "별로였음", "너무 별로", "돈 아까움", "실망", "안간다", "차라리 딴데", "속았음",
        "후회만 남음", "맘 상함", "한 번 가고 안 감", "사기템", "버림", "쓰레기템", "더러움",
        "터무니 없음", "낚임", "두 번은 안 감", "쓰레기 서비스", "불안함", "서비스 미숙", "당황스러움",
        "너무 느림", "후회만 가득", "갈 이유 없음", "마음 상함", "말도 안 됨", "너무 허접", "없어서 못감",
        "체감상 별로", "기분 상함", "하자 많음", "최악의 결정", "구매 후회", "안습", "저렴한 티", "헛된 시간",
        "망함", "지저분", "두 번 안 감", "손해봄", "싼 맛에 샀는데 후회", "기대보다 별로", "차라리 안 사는 게",
        "기분 상함", "아무것도 아님", "안 사는 게 나음", "갓성비 아님", "여기만 빼고 다 좋음", "진절머리",
        "진저리", "별로다 못해 최악", "위생이 별로"
    ]
    
    def label_review(tokens):
        if any(word in tokens for word in positive_words):
            return '긍정'
        elif any(word in tokens for word in negative_words):
            return '부정'
        else:
            return '중립'
    
    df['label'] = df['tokenized'].apply(label_review)
    
    return df, text_column

# 전처리된 데이터 저장
print("데이터 전처리가 완료되었습니다.")


# koBERT

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset

# KoBERT 모델을 이용한 감정 분석을 위한 데이터 전처리
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_len):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# 간단한 데이터 전처리
def preprocess_reviews(df):
    # 리뷰 컬럼 이름 확인 후 사용 가능한 컬럼 찾기
    possible_columns = ['리뷰', 'review', '내용', 'Review']
    text_column = None
    for column in possible_columns:
        if column in df.columns:
            text_column = column
            break
    
    if text_column is None:
        raise KeyError("리뷰 텍스트 컬럼을 찾을 수 없습니다. 컬럼 이름을 확인해주세요.")
    
    # 결측치 제거
    df = df.dropna(subset=[text_column])
    
    # 리뷰 텍스트 전처리 (간단하게 특수문자 제거 예시)
    df.loc[:, text_column] = df[text_column].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
    
    return df, text_column

# KoBERT 모델을 이용한 감정 분석 수행
def sentiment_analysis_kobert(df, text_column):
    # KoBERT 토크나이저 및 모델 불러오기
    tokenizer = BertTokenizer.from_pretrained('skt/kobert-base-v1')
    model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1')

    # 데이터셋 생성
    reviews = df[text_column].tolist()
    dataset = ReviewDataset(reviews, tokenizer, max_len=128)
    dataloader = DataLoader(dataset, batch_size=16)

    # 평가 모드로 설정
    model.eval()

    # 라벨 예측
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.tolist())

    # 라벨링 결과 저장
    df['label'] = ['긍정' if pred == 1 else '부정' for pred in predictions]
    return df

# 전처리 실행
combined_df = pd.read_csv('./data/combined_reviews.csv')
processed_df, text_column = preprocess_reviews(combined_df)

# KoBERT 감정 분석 수행
processed_df = sentiment_analysis_kobert(processed_df, text_column)

# 전처리된 데이터 저장
processed_df.to_csv("./data/processed_reviews_with_labels.csv", index=False)

print("데이터 전처리 및 라벨링이 완료되었습니다.")


#